In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


In [2]:
pip install git+https://github.com/huggingface/transformers@master

  Cloning https://github.com/huggingface/transformers (to revision master) to /tmp/pip-req-build-q_ka8en_
  Running command git clone --filter=blob:none -q https://github.com/huggingface/transformers /tmp/pip-req-build-q_ka8en_
  Resolved https://github.com/huggingface/transformers to commit 84ea427f460ffc8d2ddc08a341ccda076c24fc1f
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.15.0.dev0-py3-none-any.whl size=3360605 sha256=0c8765281bf8c1d6706f27a5620156f78762aee826bf3c1a0cf3dbffe2b151fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-_kun1986/wheels/6d/cd/9a/37a012ea494d9bdfc2930af284190c7423b1d7c8e8726c2efb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

In [4]:
!nvidia-smi

Sun Dec 19 18:23:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
torch.manual_seed(42)

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Embedding(50259, 1024)

In [7]:
descriptions = pd.read_csv('../input/netflix-shows/netflix_titles.csv')['description']
descriptions

0       As her father nears the end of his life, filmm...
1       After crossing paths at a party, a Cape Town t...
2       To protect his family from a powerful drug lor...
3       Feuds, flirtations and toilet talk go down amo...
4       In a city of coaching centers known to train I...
                              ...                        
8802    A political cartoonist, a crime reporter and a...
8803    While living alone in a spooky town, a young g...
8804    Looking to survive in a world taken over by zo...
8805    Dragged from civilian life, a former superhero...
8806    A scrappy but poor boy worms his way into a ty...
Name: description, Length: 8807, dtype: object

In [8]:
max_length = max([len(tokenizer.encode(description)) for description in descriptions])

In [9]:
class NetflixDataset(Dataset):
    def __init__(self,txt_list, tokenizer,max_length):
        self.input_ids = []
        self.attn_mask = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation =True, max_length = max_length, padding = "max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_mask.append(torch.tensor(encodings_dict['attention_mask']))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx],self.attn_mask[idx]
        

In [10]:
dataset = NetflixDataset(descriptions, tokenizer, max_length = max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size ])

In [11]:
import gc
gc.collect()

25

In [12]:
torch.cuda.empty_cache()

In [13]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [14]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

***** Running training *****
  Num examples = 7926
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 7926


Step,Training Loss
100,5.472100
200,2.029000
300,1.915800
400,1.875700
500,1.901700
600,1.838800
700,1.904300
800,1.863900
900,1.837000
1000,1.902400


Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7926, training_loss=1.8267689400960965, metrics={'train_runtime': 1161.3176, 'train_samples_per_second': 6.825, 'train_steps_per_second': 6.825, 'total_flos': 891356768944128.0, 'train_loss': 1.8267689400960965, 'epoch': 1.0})

GPT Generated Description

In [15]:
generated = tokenizer("<|startoftext|> ", return_tensors="pt").input_ids.cuda()

In [16]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [17]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0:  ̶Curious pet owner Andy Bocarski sets eyes on another species that brings the spirit of adventure and creativity, but also includes human danger. So prepare for the adventure, name all of "Roots."
1: ?????? (Seth) meets in Amsterdam with his bestest friends when mysterious messages appear that bring his favorite foods together.
2:   What happens when one falls pregnant with her husband’s help?
3:  ̶ ̶Bollywood veteran director and activist, Satya Malhotra appears today at a meeting dedicated to women empowerment in Delhi to discuss "rape myths" she encountered while traveling.
4:  ” To take revenge on a corrupt, corrupt court and to keep his own wife and children safe” in modern society, a father attempts to rob another to give to those who could take care of him.
5:  ㅋㅋThe funniest and scini-twinchiest comic dishes raucously about life and the joys of living, whether its home birth of babies or just staying at home all morning.
6:  ưö er t the other. Now, with all the romance and

Original Description (Random)

In [18]:
pd.options.display.max_colwidth = 1000
descriptions.sample(10)

4970                       Three buddies with big dreams go from underachieving slackers to badass warriors when their posh hotel is taken over by terrorists.
3362         In his first stand-up special, Arsenio Hall discusses getting older, the changing times and culture, social issues and even bothersome baby toes.
5494                                                  Music meets imagination in this inventive animated series about thinking outside the box and having fun.
1688                        Explore an array of unique competitions, from the quirky to the bizarre, and meet their passionate communities in this docuseries.
1349         From his days as a petty thief to becoming head of a drug-trafficking empire, this riveting series charts the life of the infamous Pablo Escobar.
4862        This anime adventure follows the battle between a saint of Athena and an avatar of Hades who's working on a painting that could destroy the world.
2676     A top Israeli agent comes out of reti